In [8]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mongodb import MongoDBClient
from configs import BOOTSTRAP_SERVERS, KAFKA_TOPIC, MONGO_HOST, MONGO_PORT, MONGO_DB_NAME

In [2]:
spark = SparkSession.builder \
    .appName("KafkaSparkMongoDB") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/mehrdad/anaconda3/envs/DE/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mehrdad/.ivy2/cache
The jars for the packages stored in: /home/mehrdad/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0ca819a-2661-439e-b091-1db93b397d67;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;

In [3]:
schema = StructType([
    StructField("key", StringType(), True),
    StructField("value", StringType(), True),
    StructField("timestamp", StringType(), True)
])

In [4]:
client = MongoDBClient(MONGO_HOST, MONGO_PORT, MONGO_DB_NAME, username='admin', password='password')


In [14]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("startingOffsets", "earliest") \
    .option("subscribe", "subs") \
    .load()

In [31]:
parsed_df = df \
    .select(from_json(col("value").cast("string"), schema).alias("data")) \
    .select("data.*")

In [34]:
show_value = lambda x: print(x)

In [38]:
processed_df = parsed_df \
    .withColumn("processed_value", show_value(col("value")))

Column<'value'>


PySparkTypeError: [NOT_COLUMN] Argument `col` should be a Column, got NoneType.

In [37]:
parsed_df

DataFrame[key: string, value: string, timestamp: string]

In [43]:
def process_batch(df, epoch_id):
    if not df.isEmpty():
    # Convert DataFrame to RDD
        rdd = df.rdd
    if not rdd.isEmpty():
    # Process each message in the RDD
        for row in rdd.collect():
    # Example: assume each Kafka message is a JSON document
    # Insert the document into MongoDB
            print(row.asDict())
    # collection.insert_one(row.asDict())

In [46]:
query = df.writeStream.foreachBatch(process_batch).start()

23/08/26 21:03:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-771038dc-1627-4b48-afba-1509cbb2a1f8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/08/26 21:03:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/08/26 21:03:38 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/08/26 21:03:38 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/08/26 21:03:38 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/08/26 21:03:38 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known con

{'key': None, 'value': bytearray(b'""'), 'topic': 'subs', 'partition': 0, 'offset': 0, 'timestamp': datetime.datetime(2023, 8, 26, 19, 32, 22, 752000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"wwwaws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 1, 'timestamp': datetime.datetime(2023, 8, 26, 19, 32, 22, 874000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 2, 'timestamp': datetime.datetime(2023, 8, 26, 19, 32, 22, 880000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"wwwqa2.aws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 3, 'timestamp': datetime.datetime(2023, 8, 26, 19, 32, 22, 890000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"china.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 4, 'timestamp': datetime.datetime(2023, 8, 26, 19, 32, 22, 894000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"m.caterpillar.com"')

In [ ]:
print('.')

23/08/26 21:09:55 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:09:56 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:09:56 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:09:56 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:09:56 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:09:56 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

Row(key=None, value=bytearray(b'""'), topic='subs', partition=0, offset=39, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 794000), timestampType=0)
Row(key=None, value=bytearray(b'"wwwdev.aws.caterpillar.com"'), topic='subs', partition=0, offset=40, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 819000), timestampType=0)
Row(key=None, value=bytearray(b'"reports.caterpillar.com"'), topic='subs', partition=0, offset=41, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 832000), timestampType=0)
{'key': None, 'value': bytearray(b'""'), 'topic': 'subs', 'partition': 0, 'offset': 39, 'timestamp': datetime.datetime(2023, 8, 26, 21, 9, 52, 794000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"wwwdev.aws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 40, 'timestamp': datetime.datetime(2023, 8, 26, 21, 9, 52, 819000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"reports.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 41, 'timesta

23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:10:00 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

Row(key=None, value=bytearray(b'"china.caterpillar.com"'), topic='subs', partition=0, offset=42, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 836000), timestampType=0)
Row(key=None, value=bytearray(b'"w.caterpillar.com"'), topic='subs', partition=0, offset=43, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 840000), timestampType=0)
Row(key=None, value=bytearray(b'"ay4omq1chacs-www.caterpillar.com"'), topic='subs', partition=0, offset=44, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 852000), timestampType=0)
Row(key=None, value=bytearray(b'"0q91frjc734c-wwwqaext.caterpillar.com"'), topic='subs', partition=0, offset=45, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 861000), timestampType=0)
Row(key=None, value=bytearray(b'"strategy.caterpillar.com"'), topic='subs', partition=0, offset=46, timestamp=datetime.datetime(2023, 8, 26, 21, 9, 52, 884000), timestampType=0)
Row(key=None, value=bytearray(b'"dealertechjobs.caterpillar.com"'), topic='subs', partition=0, offs

23/08/26 21:11:08 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:09 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:09 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:09 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:09 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:10 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

{'key': None, 'value': bytearray(b'"investor.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 66, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 796000), 'timestampType': 0}
Row(key=None, value=bytearray(b'"investor.caterpillar.com"'), topic='subs', partition=0, offset=66, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 796000), timestampType=0)
Row(key=None, value=bytearray(b'"2a.caterpillar.com"'), topic='subs', partition=0, offset=67, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 801000), timestampType=0)
Row(key=None, value=bytearray(b'"staging-www.aws.caterpillar.com"'), topic='subs', partition=0, offset=68, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 810000), timestampType=0)
{'key': None, 'value': bytearray(b'"investor.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 66, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 796000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"2a.caterpillar.com"'), 'topic': 'subs

23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

{'key': None, 'value': bytearray(b'"2a.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 67, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 801000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"staging-www.aws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 68, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 810000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"content.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 69, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 819000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"aws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 70, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 826000), 'timestampType': 0}
{'key': None, 'value': bytearray(b'"www.aws.caterpillar.com"'), 'topic': 'subs', 'partition': 0, 'offset': 71, 'timestamp': datetime.datetime(2023, 8, 26, 21, 11, 7, 830000), 'timestampType': 0}
{'key': None, 'value': byt

23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/08/26 21:11:12 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

Row(key=None, value=bytearray(b'"content.caterpillar.com"'), topic='subs', partition=0, offset=69, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 819000), timestampType=0)
Row(key=None, value=bytearray(b'"aws.caterpillar.com"'), topic='subs', partition=0, offset=70, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 826000), timestampType=0)
Row(key=None, value=bytearray(b'"www.aws.caterpillar.com"'), topic='subs', partition=0, offset=71, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 830000), timestampType=0)
Row(key=None, value=bytearray(b'"caterpillar.com.caterpillar.com"'), topic='subs', partition=0, offset=72, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 836000), timestampType=0)
Row(key=None, value=bytearray(b'"ir.caterpillar.com"'), topic='subs', partition=0, offset=73, timestamp=datetime.datetime(2023, 8, 26, 21, 11, 7, 839000), timestampType=0)
Row(key=None, value=bytearray(b'"com.caterpillar.com"'), topic='subs', partition=0, offset=74, timestamp=datetime.da

In [48]:
a = "com.caterpillar.com"

In [50]:
a.split('.caterpillar.com')

['com', '']

23/08/26 22:03:51 WARN NetworkClient: [AdminClient clientId=adminclient-3] Connection to node 1001 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/08/26 22:03:51 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/08/26 22:03:52 WARN NetworkClient: [AdminClient clientId=adminclient-2] Connection to node 1001 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/08/26 22:03:52 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/08/26 22:03:52 WARN NetworkClient: [AdminClient clientId=adminclient-3] Connection to node 1001 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
23/08/26 22:03:53 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1